In [18]:
from cv2 import AGAST_FEATURE_DETECTOR_AGAST_5_8
from src.getter import Get
from models.sqliteModels import _insert_companies
import os

# create a Get object
get = Get()

# run if name == main script
df = get.companies_df()

c:\Users\charl\OneDrive\dev\source\finlearn\src\getter.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _companies_df = NYSE_df.append(NASDAQ_df)
c:\Users\charl\OneDrive\dev\source\finlearn\src\getter.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _companies_df = _companies_df.append(AMEX_df)
c:\Users\charl\OneDrive\dev\source\finlearn\src\getter.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _companies_df = _companies_df.append(OTCBB_df)


      exchange symbol                               name
0         NYSE      A               Agilent Technologies
1         NYSE     AA                         Alcoa Corp
2         NYSE    AAC         Ares Acquisition Corp Cl A
3         NYSE    AAI    Arlington Asset Investment Corp
4         NYSE   AAIN  Arlington Asset Investment Corp 6
...        ...    ...                                ...
19395    OTCBB  ZURVY      Zurich Insurance Group Ag ADR
19396    OTCBB   ZVLO                          Esoft Inc
19397    OTCBB  ZZHGF  Zhongan Online P & C Insurance Co
19398    OTCBB   ZZLL    Zzll Information Technology Inc
19399    OTCBB  ZZZOF             Zinc One Resources Inc

[19400 rows x 3 columns]


data/marketdata.db


In [85]:

from models.sqliteModels import _query_symbols


def yield_symbols():
    # table_name
    data = _query_symbols()
    symbols = [stock[0] for stock in data]
    for symbol in symbols: yield symbol

In [86]:

    stocks = yield_symbols(db_path)

In [87]:
def get_next_stock(stock:list):
    """
    :param stocks: list of stocks to yield, one at a time, so that 
    the other functions can execute their processes (getting price history, 
    quote data, fundamentals, and perform technical analysis)
    """
    symbol = next(stock)
    print(symbol)

In [ ]:
# create the quote_df to pass to the already created functions
print(f"[1.3] Retrieve data from DataFrame's symbol column")
_stocks = companies_df["symbol"]

# #####################################################################
# CREATE COMPANIES LIST:
#####################################################################
# --->[6] Create Chunked list
stock_chunks = list(get.chunks(list(set(_stocks)), 200))

# Give quote and fundamental data an variable to save to:
# Get Quote Data
print("[3] Retrieve Quote and Fundamental data..")
quote_data = pd.concat([get.quotes(each) for each in stock_chunks])
print("[3.1] Quote Data Retrieved")

# Get Fundamental Data
fundamental_data = pd.concat([get.fundamentals(each)
                              for each in stock_chunks])
print("[3.2] Fundamental Data Retrieved")

print("[4] Insert Quote and Fundamental Data")
# Insert the fundamental and quote data to the database:
insert_quote_and_fundamental_data(quote_data=quote_data,
                                  fundamental_data=fundamental_data, db=db)

# print(quote_data)
